In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.optimize as opt
from mpl_toolkits.mplot3d import Axes3D
from trajectory_visualization import plot_trajectory, sidebyside, shift_trajectory, overlay, shift_trajectory3D
from trajectory_visualization import plot_trajectories3D, plot_3Doverlay, plot_MSDorDeff, plot_MeanMSDorDeff, randtraj, multrandtraj
from trajectory_visualization import randtraj2, plot_Mean2DMSDsorDeff, plot_MSDorDeffLR, LRfor3D2D
from operator import itemgetter
from random_traj import randaniso, randconv, multrandaniso, multrandconv, randsamp, multrandsamp, randall, multrandall
from random_traj import randanisorot, multrandanisorot
from ellipsoid import maxtraj, mvee, plot_mvee, plot_mveeoverl, rotmat, rotmat2, plot_mveeoverl2
from ellipsoid import ellipsax, MSDS, andiff, plot_anisoMSDS, plot_anisodiff
import random
import numpy as np
import numpy.linalg as la
from mpl_toolkits.mplot3d import Axes3D
from ellipsoid import extrema, enclosed_MSD

pi = np.pi
sin = np.sin
cos = np.cos

In [23]:
trajectory = np.genfromtxt('../sample_data/Slice2_2DVideo_4_22.csv',
            delimiter =",")

#Remove titles from columns
trajectory=np.delete(trajectory, 0,0)
#Remove the number column from dataset
trajectory=np.delete(trajectory,0,1)

In [24]:
    """
    Output:
    0 particle number
    1 frames
    2 x coordinate
    3 y coordinate
    4 z coordinate
    5 centered x coordinate
    6 centered y coordinate
    7 centered z coordinate
    8 3D MSD 
    9 2D xy MSD
    10 2D xz MSD
    11 2D yz MSD
    12 1D x MSD
    13 1D y MSD
    14 1D z MSD
    15 time
    16 3D Deff
    17 2D xy Deff
    18 2D xz Deff
    19 2D yz Deff
    20 1D x Deff
    21 1D y Deff
    22 1D z Deff
    """

'\nOutput:\n0 particle number\n1 frames\n2 x coordinate\n3 y coordinate\n4 z coordinate\n5 centered x coordinate\n6 centered y coordinate\n7 centered z coordinate\n8 3D MSD \n9 2D xy MSD\n10 2D xz MSD\n11 2D yz MSD\n12 1D x MSD\n13 1D y MSD\n14 1D z MSD\n15 time\n16 3D Deff\n17 2D xy Deff\n18 2D xz Deff\n19 2D yz Deff\n20 1D x Deff\n21 1D y Deff\n22 1D z Deff\n'

In [25]:
"""
# Creating a function to fill in blanks
x = dict()
y = dict()
z = dict()

xc = dict()
yc = dict()
zc = dict()

particles = ttraject[:, 0]
total = int(max(particles))
total1 = total + 1
rawx = ttraject[:, 5]
rawy = ttraject[:, 6]
rawz = ttraject[:, 7]

# Creates an array for each trajectory containing all xyz data
for num in range(1, total1):

    hold = np.where(particles == num)
    itindex = hold[0]
    min1 = min(itindex)
    max1 = max(itindex)
    x[num] = (rawx[min1:max1])
    y[num] = (rawy[min1:max1])
    z[num] = (rawz[min1:max1])
    
    xc[num] = x[num] - ((max(x[num])+min(x[num]))/2)
    yc[num] = y[num] - ((max(y[num])+min(y[num]))/2)
    zc[num] = z[num] - ((max(z[num])+min(z[num]))/2)
"""

'\n# Creating a function to fill in blanks\nx = dict()\ny = dict()\nz = dict()\n\nxc = dict()\nyc = dict()\nzc = dict()\n\nparticles = ttraject[:, 0]\ntotal = int(max(particles))\ntotal1 = total + 1\nrawx = ttraject[:, 5]\nrawy = ttraject[:, 6]\nrawz = ttraject[:, 7]\n\n# Creates an array for each trajectory containing all xyz data\nfor num in range(1, total1):\n\n    hold = np.where(particles == num)\n    itindex = hold[0]\n    min1 = min(itindex)\n    max1 = max(itindex)\n    x[num] = (rawx[min1:max1])\n    y[num] = (rawy[min1:max1])\n    z[num] = (rawz[min1:max1])\n    \n    xc[num] = x[num] - ((max(x[num])+min(x[num]))/2)\n    yc[num] = y[num] - ((max(y[num])+min(y[num]))/2)\n    zc[num] = z[num] - ((max(z[num])+min(z[num]))/2)\n'

In [26]:
"""
dataset = dict()

particles = trajectory[:, 0]
total = int(max(particles))
total1 = total + 1
rawdataset = trajectory[:, :]

# Creates an array for each trajectory containing all xyz data
for num in range(1, total1):

    hold = np.where(particles == num)
    itindex = hold[0]
    min1 = min(itindex)
    max1 = max(itindex)
    dataset[num] = (rawdataset[min1:max1, 0:5])
"""

'\ndataset = dict()\n\nparticles = trajectory[:, 0]\ntotal = int(max(particles))\ntotal1 = total + 1\nrawdataset = trajectory[:, :]\n\n# Creates an array for each trajectory containing all xyz data\nfor num in range(1, total1):\n\n    hold = np.where(particles == num)\n    itindex = hold[0]\n    min1 = min(itindex)\n    max1 = max(itindex)\n    dataset[num] = (rawdataset[min1:max1, 0:5])\n'

In [27]:
#Original code.  Changed to a function below.

#I will now try to build a separate array from dataset rather than filling in missing pieces.
#This piece now works.  I will eventually build this into a function to do this for all datasets.  
#This works on datasets from the MOSAIC code.

"""
shap = int(max(dataset[2][:,1])) +1 
filledin = np.zeros((shap,5))
filledin[0,:] = dataset[2][0,:]

count = 0
new = 0 
other = 0
tot = 0

for num in range(1, shap):
    if dataset[2][num-new,1]-dataset[2][num-new-1,1]==1:
        count = count + 1
        filledin[num, :] = dataset[2][num-new, :]
    elif dataset[2][num - new,1]-dataset[2][num - new -1,1]>1:
        new = new + 1
        iba = int(dataset[2][num - new+1,1]-dataset[2][num - new,1])
        togoin = dataset[2][num - new]
        togoin[1] = togoin[1] + 1
        filledin[num, :] = togoin
        #dataset[2] = np.insert(dataset[2], [num + new - 2], togoin, axis=0)
        
    else:
        other = other + 1
    tot = count + new + other
    
dataset[2] = filledin
"""

'\nshap = int(max(dataset[2][:,1])) +1 \nfilledin = np.zeros((shap,5))\nfilledin[0,:] = dataset[2][0,:]\n\ncount = 0\nnew = 0 \nother = 0\ntot = 0\n\nfor num in range(1, shap):\n    if dataset[2][num-new,1]-dataset[2][num-new-1,1]==1:\n        count = count + 1\n        filledin[num, :] = dataset[2][num-new, :]\n    elif dataset[2][num - new,1]-dataset[2][num - new -1,1]>1:\n        new = new + 1\n        iba = int(dataset[2][num - new+1,1]-dataset[2][num - new,1])\n        togoin = dataset[2][num - new]\n        togoin[1] = togoin[1] + 1\n        filledin[num, :] = togoin\n        #dataset[2] = np.insert(dataset[2], [num + new - 2], togoin, axis=0)\n        \n    else:\n        other = other + 1\n    tot = count + new + other\n    \ndataset[2] = filledin\n'

In [28]:
def fillin(data):
    """
    This function is perfect.  It shifts the frames by the startframe and fills in any blank frames.
    """
    def startshift(data1):
        startframe = data1[0, 1]
        data1[:, 1] = data1[:, 1] - startframe
        return data1
    
    data = startshift(data)

    shap = int(max(data[:,1])) +1 
    filledin = np.zeros((shap,5))
    filledin[0,:] = data[0,:]

    count = 0
    new = 0 
    other = 0
    tot = 0

    for num in range(1, shap):
        if data[num-new,1]-data[num-new-1,1]==1:
            count = count + 1
            filledin[num, :] = data[num-new, :]
        elif data[num - new,1]-data[num - new -1,1]>1:
            new = new + 1
            iba = int(data[num - new+1,1]-data[num - new,1])
            togoin = data[num - new]
            togoin[1] = togoin[1] + 1
            filledin[num, :] = togoin
            #dataset[2] = np.insert(dataset[2], [num + new - 2], togoin, axis=0)
        
        else:
            other = other + 1
        tot = count + new + other
    
    return filledin

In [29]:
#I realized I also will need a function to shift the datasets for those datasets that don't start at frame 0.

"""
def startshift(data1):
    startframe = data1[0, 1]
    data1[:, 1] = data1[:, 1] - startframe
"""   

'\ndef startshift(data1):\n    startframe = data1[0, 1]\n    data1[:, 1] = data1[:, 1] - startframe\n'

In [30]:
"""
xmax = max(two[:, 2])
xmin = min(two[:, 2])
ymax = max(two[:, 3])
ymin = min(two[:, 3])
zmax = max(two[:, 4])
zmin = min(two[:, 4])


xc = np.array([two[:, 2] - ((xmax+xmin)/2)])
yc = np.array([two[:, 3] - ((ymax+ymin)/2)])
zc = np.array([two[:, 4] - ((zmax+zmin)/2)])

xstart = xc[0, 0]
ystart = yc[0, 0]
zstart = zc[0, ]
"""

'\nxmax = max(two[:, 2])\nxmin = min(two[:, 2])\nymax = max(two[:, 3])\nymin = min(two[:, 3])\nzmax = max(two[:, 4])\nzmin = min(two[:, 4])\n\n\nxc = np.array([two[:, 2] - ((xmax+xmin)/2)])\nyc = np.array([two[:, 3] - ((ymax+ymin)/2)])\nzc = np.array([two[:, 4] - ((zmax+zmin)/2)])\n\nxstart = xc[0, 0]\nystart = yc[0, 0]\nzstart = zc[0, ]\n'

In [31]:
"""
two = np.append(two, xc.T, axis=1)
two = np.append(two, yc.T, axis=1)
two = np.append(two, zc.T, axis=1)
two
"""

'\ntwo = np.append(two, xc.T, axis=1)\ntwo = np.append(two, yc.T, axis=1)\ntwo = np.append(two, zc.T, axis=1)\ntwo\n'

In [32]:
"""
the = two.shape[0]
MSD3 = np.zeros((the,1))
M2xy = np.zeros((the,1))
M2xz = np.zeros((the,1))
M2yz = np.zeros((the,1))
M1x = np.zeros((the,1))
M1y = np.zeros((the,1))
M1z = np.zeros((the,1))

time = 0.25 * two[:, 1]
time[0] = 0.0000000001
time1 = np.array([time]).T

D3 = np.zeros((the,1))
D2xy = np.zeros((the,1))
D2xz = np.zeros((the,1))
D2yz = np.zeros((the,1))
D1x = np.zeros((the,1))
D1y = np.zeros((the,1))
D1z = np.zeros((the,1))

for num in range(0, the):
    MSD3[num, 0] = (xc[0, num] - xstart)**2 + (yc[0, num] - ystart)**2 + (zc[0, num] - zstart)**2
    M2xy[num, 0] = (xc[0, num] - xstart)**2 + (yc[0, num] - ystart)**2
    M2xz[num, 0] = (xc[0, num] - xstart)**2 + (zc[0, num] - zstart)**2
    M2yz[num, 0] = (yc[0, num] - ystart)**2 + (zc[0, num] - zstart)**2
    M1x[num, 0] = (xc[0, num] - xstart)**2
    M1y[num, 0] = (yc[0, num] - ystart)**2
    M1z[num, 0] = (zc[0, num] - zstart)**2
    
    D3[num, 0] = MSD3[num, 0]/(6*time[num])
    D2xy[num, 0] = M2xy[num, 0]/(4*time[num])
    D2xz[num, 0] = M2xz[num, 0]/(4*time[num])
    D2yz[num, 0] = M2yz[num, 0]/(4*time[num])
    D1x[num, 0] = M1x[num, 0]/(2*time[num])
    D1y[num, 0] = M1y[num, 0]/(2*time[num])
    D1z[num, 0] = M1z[num, 0]/(2*time[num])
"""

'\nthe = two.shape[0]\nMSD3 = np.zeros((the,1))\nM2xy = np.zeros((the,1))\nM2xz = np.zeros((the,1))\nM2yz = np.zeros((the,1))\nM1x = np.zeros((the,1))\nM1y = np.zeros((the,1))\nM1z = np.zeros((the,1))\n\ntime = 0.25 * two[:, 1]\ntime[0] = 0.0000000001\ntime1 = np.array([time]).T\n\nD3 = np.zeros((the,1))\nD2xy = np.zeros((the,1))\nD2xz = np.zeros((the,1))\nD2yz = np.zeros((the,1))\nD1x = np.zeros((the,1))\nD1y = np.zeros((the,1))\nD1z = np.zeros((the,1))\n\nfor num in range(0, the):\n    MSD3[num, 0] = (xc[0, num] - xstart)**2 + (yc[0, num] - ystart)**2 + (zc[0, num] - zstart)**2\n    M2xy[num, 0] = (xc[0, num] - xstart)**2 + (yc[0, num] - ystart)**2\n    M2xz[num, 0] = (xc[0, num] - xstart)**2 + (zc[0, num] - zstart)**2\n    M2yz[num, 0] = (yc[0, num] - ystart)**2 + (zc[0, num] - zstart)**2\n    M1x[num, 0] = (xc[0, num] - xstart)**2\n    M1y[num, 0] = (yc[0, num] - ystart)**2\n    M1z[num, 0] = (zc[0, num] - zstart)**2\n    \n    D3[num, 0] = MSD3[num, 0]/(6*time[num])\n    D2xy[num,

In [33]:
"""two = np.append(two, MSD3, axis=1)
two = np.append(two, M2xy, axis=1)
two = np.append(two, M2xz, axis=1)
two = np.append(two, M2yz, axis=1)
two = np.append(two, M1x, axis=1)
two = np.append(two, M1y, axis=1)
two = np.append(two, M1z, axis=1)
two = np.append(two, MSD3, axis=1)
two = np.append(two, time1, axis=1)
two = np.append(two, D3, axis=1)
two = np.append(two, D2xy, axis=1)
two = np.append(two, D2xz, axis=1)
two = np.append(two, D2yz, axis=1)
two = np.append(two, D1x, axis=1)
two = np.append(two, D1y, axis=1)
two = np.append(two, D1z, axis=1)
"""

'two = np.append(two, MSD3, axis=1)\ntwo = np.append(two, M2xy, axis=1)\ntwo = np.append(two, M2xz, axis=1)\ntwo = np.append(two, M2yz, axis=1)\ntwo = np.append(two, M1x, axis=1)\ntwo = np.append(two, M1y, axis=1)\ntwo = np.append(two, M1z, axis=1)\ntwo = np.append(two, MSD3, axis=1)\ntwo = np.append(two, time1, axis=1)\ntwo = np.append(two, D3, axis=1)\ntwo = np.append(two, D2xy, axis=1)\ntwo = np.append(two, D2xz, axis=1)\ntwo = np.append(two, D2yz, axis=1)\ntwo = np.append(two, D1x, axis=1)\ntwo = np.append(two, D1y, axis=1)\ntwo = np.append(two, D1z, axis=1)\n'

In [34]:
#Builds a new dictionary based on the dictionary defined above (dataset).  Dataset itself
#is defined by the input csv file from the MOSIAC package and breaks down the dataset by
# particle.  This code builds MSDs and Deffs onto the datasets.  

"""
flee = dict()
for num in range(1, total1):
    flee[num] = fillin(dataset[num])
    
    xmax = max(flee[num][:, 2])
    xmin = min(flee[num][:, 2])
    ymax = max(flee[num][:, 3])
    ymin = min(flee[num][:, 3])
    zmax = max(flee[num][:, 4])
    zmin = min(flee[num][:, 4])


    xc = np.array([flee[num][:, 2] - ((xmax+xmin)/2)])
    yc = np.array([flee[num][:, 3] - ((ymax+ymin)/2)])
    zc = np.array([flee[num][:, 4] - ((zmax+zmin)/2)])

    xstart = xc[0, 0]
    ystart = yc[0, 0]
    zstart = zc[0, 0]
    
    flee[num] = np.append(flee[num], xc.T, axis=1)
    flee[num] = np.append(flee[num], yc.T, axis=1)
    flee[num] = np.append(flee[num], zc.T, axis=1)
    
    the = flee[num].shape[0]
    MSD3 = np.zeros((the,1))
    M2xy = np.zeros((the,1))
    M2xz = np.zeros((the,1))
    M2yz = np.zeros((the,1))
    M1x = np.zeros((the,1))
    M1y = np.zeros((the,1))
    M1z = np.zeros((the,1))

    time = 0.25 * flee[num][:, 1]
    time[0] = 0.0000000001
    time1 = np.array([time]).T

    D3 = np.zeros((the,1))
    D2xy = np.zeros((the,1))
    D2xz = np.zeros((the,1))
    D2yz = np.zeros((the,1))
    D1x = np.zeros((the,1))
    D1y = np.zeros((the,1))
    D1z = np.zeros((the,1))

    for bum in range(0, the):
        MSD3[bum, 0] = (xc[0, bum] - xstart)**2 + (yc[0, bum] - ystart)**2 + (zc[0, bum] - zstart)**2
        M2xy[bum, 0] = (xc[0, bum] - xstart)**2 + (yc[0, bum] - ystart)**2
        M2xz[bum, 0] = (xc[0, bum] - xstart)**2 + (zc[0, bum] - zstart)**2
        M2yz[bum, 0] = (yc[0, bum] - ystart)**2 + (zc[0, bum] - zstart)**2
        M1x[bum, 0] = (xc[0, bum] - xstart)**2
        M1y[bum, 0] = (yc[0, bum] - ystart)**2
        M1z[bum, 0] = (zc[0, bum] - zstart)**2
    
        D3[bum, 0] = MSD3[bum, 0]/(6*time[bum])
        D2xy[bum, 0] = M2xy[bum, 0]/(4*time[bum])
        D2xz[bum, 0] = M2xz[bum, 0]/(4*time[bum])
        D2yz[bum, 0] = M2yz[bum, 0]/(4*time[bum])
        D1x[bum, 0] = M1x[bum, 0]/(2*time[bum])
        D1y[bum, 0] = M1y[bum, 0]/(2*time[bum])
        D1z[bum, 0] = M1z[bum, 0]/(2*time[bum])
    
    flee[num] = np.append(flee[num], MSD3, axis=1)
    flee[num] = np.append(flee[num], M2xy, axis=1)
    flee[num] = np.append(flee[num], M2xz, axis=1)
    flee[num] = np.append(flee[num], M2yz, axis=1)
    flee[num] = np.append(flee[num], M1x, axis=1)
    flee[num] = np.append(flee[num], M1y, axis=1)
    flee[num] = np.append(flee[num], M1z, axis=1)
    flee[num] = np.append(flee[num], time1, axis=1)
    flee[num] = np.append(flee[num], D3, axis=1)
    flee[num] = np.append(flee[num], D2xy, axis=1)
    flee[num] = np.append(flee[num], D2xz, axis=1)
    flee[num] = np.append(flee[num], D2yz, axis=1)
    flee[num] = np.append(flee[num], D1x, axis=1)
    flee[num] = np.append(flee[num], D1y, axis=1)
    flee[num] = np.append(flee[num], D1z, axis=1)
"""

'\nflee = dict()\nfor num in range(1, total1):\n    flee[num] = fillin(dataset[num])\n    \n    xmax = max(flee[num][:, 2])\n    xmin = min(flee[num][:, 2])\n    ymax = max(flee[num][:, 3])\n    ymin = min(flee[num][:, 3])\n    zmax = max(flee[num][:, 4])\n    zmin = min(flee[num][:, 4])\n\n\n    xc = np.array([flee[num][:, 2] - ((xmax+xmin)/2)])\n    yc = np.array([flee[num][:, 3] - ((ymax+ymin)/2)])\n    zc = np.array([flee[num][:, 4] - ((zmax+zmin)/2)])\n\n    xstart = xc[0, 0]\n    ystart = yc[0, 0]\n    zstart = zc[0, 0]\n    \n    flee[num] = np.append(flee[num], xc.T, axis=1)\n    flee[num] = np.append(flee[num], yc.T, axis=1)\n    flee[num] = np.append(flee[num], zc.T, axis=1)\n    \n    the = flee[num].shape[0]\n    MSD3 = np.zeros((the,1))\n    M2xy = np.zeros((the,1))\n    M2xz = np.zeros((the,1))\n    M2yz = np.zeros((the,1))\n    M1x = np.zeros((the,1))\n    M1y = np.zeros((the,1))\n    M1z = np.zeros((the,1))\n\n    time = 0.25 * flee[num][:, 1]\n    time[0] = 0.000000000

In [35]:
#Now I will build two separate functions to act on the dictionary flee defined above.
# I will make one function to filter out any entries less than a specified number
# of columns (say, 50).  I will also build another function to cut down datasets
# so they are all the same length-- say, 50.  

"""
teancum = dict()
fifties = 0
nones = 0
cutoff = 50

for num in range(1, total1):
    if flee[num].shape[0] < cutoff:
        nones = nones + 1
    else:
        fifties = fifties + 1
        teancum[num - nones] = flee[num]
        # I must also redefine the particle numbers to reflect the new set.
        teancum[num - nones][:, 0] = fifties
"""

'\nteancum = dict()\nfifties = 0\nnones = 0\ncutoff = 50\n\nfor num in range(1, total1):\n    if flee[num].shape[0] < cutoff:\n        nones = nones + 1\n    else:\n        fifties = fifties + 1\n        teancum[num - nones] = flee[num]\n        # I must also redefine the particle numbers to reflect the new set.\n        teancum[num - nones][:, 0] = fifties\n'

In [36]:
# Now for the second function: cutting down datasets to a specified value.

"""
moroni = dict()
limit = 50

for num in range(1, fifties):
    moroni[num] = teancum[num][0:limit, :]
"""

'\nmoroni = dict()\nlimit = 50\n\nfor num in range(1, fifties):\n    moroni[num] = teancum[num][0:limit, :]\n'

In [37]:
#Now to build all this into a single function?

def prettify(traj, cut, lim):
    """
    traj: a dataset from the MOSAIC code with the top row and first column removed.
    cut: the minimum number of frames required to be included in final dataset
    lim: the specified number of frames to be included in final dataset (often the same as cut)
    """
    
    dataset = dict()

    particles = traj[:, 0]
    total = int(max(particles))
    total1 = total + 1
    rawdataset = traj[:, :]

    # Creates an array for each trajectory containing all xyz data
    for num in range(1, total1):

        hold = np.where(particles == num)
        itindex = hold[0]
        min1 = min(itindex)
        max1 = max(itindex)
        dataset[num] = (rawdataset[min1:max1, 0:5])
    
    flee = dict()
    for num in range(1, total1):
        flee[num] = fillin(dataset[num])
    
        xmax = max(flee[num][:, 2])
        xmin = min(flee[num][:, 2])
        ymax = max(flee[num][:, 3])
        ymin = min(flee[num][:, 3])
        zmax = max(flee[num][:, 4])
        zmin = min(flee[num][:, 4])


        xc = np.array([flee[num][:, 2] - ((xmax+xmin)/2)])
        yc = np.array([flee[num][:, 3] - ((ymax+ymin)/2)])
        zc = np.array([flee[num][:, 4] - ((zmax+zmin)/2)])

        xstart = xc[0, 0]
        ystart = yc[0, 0]
        zstart = zc[0, 0]
    
        flee[num] = np.append(flee[num], xc.T, axis=1)
        flee[num] = np.append(flee[num], yc.T, axis=1)
        flee[num] = np.append(flee[num], zc.T, axis=1)
    
        the = flee[num].shape[0]
        MSD3 = np.zeros((the,1))
        M2xy = np.zeros((the,1))
        M2xz = np.zeros((the,1))
        M2yz = np.zeros((the,1))
        M1x = np.zeros((the,1))
        M1y = np.zeros((the,1))
        M1z = np.zeros((the,1))
        
        #This defines the units of time.  This is more approrpriately an input to the function.  Will fix.
        time = 0.25 * flee[num][:, 1]
        time[0] = 0.0000000001
        time1 = np.array([time]).T

        D3 = np.zeros((the,1))
        D2xy = np.zeros((the,1))
        D2xz = np.zeros((the,1))
        D2yz = np.zeros((the,1))
        D1x = np.zeros((the,1))
        D1y = np.zeros((the,1))
        D1z = np.zeros((the,1))

        for bum in range(0, the):
            MSD3[bum, 0] = (xc[0, bum] - xstart)**2 + (yc[0, bum] - ystart)**2 + (zc[0, bum] - zstart)**2
            M2xy[bum, 0] = (xc[0, bum] - xstart)**2 + (yc[0, bum] - ystart)**2
            M2xz[bum, 0] = (xc[0, bum] - xstart)**2 + (zc[0, bum] - zstart)**2
            M2yz[bum, 0] = (yc[0, bum] - ystart)**2 + (zc[0, bum] - zstart)**2
            M1x[bum, 0] = (xc[0, bum] - xstart)**2
            M1y[bum, 0] = (yc[0, bum] - ystart)**2
            M1z[bum, 0] = (zc[0, bum] - zstart)**2
    
            D3[bum, 0] = MSD3[bum, 0]/(6*time[bum])
            D2xy[bum, 0] = M2xy[bum, 0]/(4*time[bum])
            D2xz[bum, 0] = M2xz[bum, 0]/(4*time[bum])
            D2yz[bum, 0] = M2yz[bum, 0]/(4*time[bum])
            D1x[bum, 0] = M1x[bum, 0]/(2*time[bum])
            D1y[bum, 0] = M1y[bum, 0]/(2*time[bum])
            D1z[bum, 0] = M1z[bum, 0]/(2*time[bum])
    
        flee[num] = np.append(flee[num], MSD3, axis=1)
        flee[num] = np.append(flee[num], M2xy, axis=1)
        flee[num] = np.append(flee[num], M2xz, axis=1)
        flee[num] = np.append(flee[num], M2yz, axis=1)
        flee[num] = np.append(flee[num], M1x, axis=1)
        flee[num] = np.append(flee[num], M1y, axis=1)
        flee[num] = np.append(flee[num], M1z, axis=1)
        flee[num] = np.append(flee[num], time1, axis=1)
        flee[num] = np.append(flee[num], D3, axis=1)
        flee[num] = np.append(flee[num], D2xy, axis=1)
        flee[num] = np.append(flee[num], D2xz, axis=1)
        flee[num] = np.append(flee[num], D2yz, axis=1)
        flee[num] = np.append(flee[num], D1x, axis=1)
        flee[num] = np.append(flee[num], D1y, axis=1)
        flee[num] = np.append(flee[num], D1z, axis=1)
    
    teancum = dict()
    fifties = 0
    nones = 0
    cutoff = cut

    for num in range(1, total1):
        if flee[num].shape[0] < cutoff:
            nones = nones + 1
        else:
            fifties = fifties + 1
            teancum[num - nones] = flee[num]
            # I must also redefine the particle numbers to reflect the new set.
            teancum[num - nones][:, 0] = fifties
    
    moroni = dict()
    limit = lim

    for num in range(1, fifties):
        moroni[num] = teancum[num][0:limit, :]
    
    return (moroni, fifties)

In [38]:
(final, tots) = prettify(trajectory, 100, 100)
tots

20

In [39]:
#It seems counterintuitive, but this is the way my code works.  Now that I built datasets for separate trajectories,
# I need to merge them into a single array to use in the code I developed for plotting.
# I based this code off of multrandtraj.

parts = tots - 1
lehi = final[1]
counter = 1

while counter < parts:
    counter = counter + 1
    lehi = np.append(lehi, final[counter], axis=0)


In [41]:
# Now for a plotting function, as modified from Testing_Tracking_Software

# Creates an array 'particles' that contains the particle number at each frame.
particles = lehi[:, 0]
position = lehi[:, 5:5+4]
total = int(max(particles))
total1 = total + 1
path = dict()

# Creates an array for each trajectory containing all xyz data
for num in range(1, total1):

    hold = np.where(particles == num)
    itindex = hold[0]
    min1 = min(itindex)
    max1 = max(itindex)
    path[num] = (position[min1:max1, :])

# Creates figure
fig = plt.figure(figsize=(24, 18), dpi=80)
ax = fig.add_subplot(111)
# ax.set_title('Particle Trajectories', x=0.5, y=1.15)

# Plots individual trajectories
for num in range(1, total1):

    ax.plot(path[num][:, 0], path[num][:, 1], label='Particle {}'.format(num))

axbox = ax.get_position()

legunit = max(lehi[:,0])
ax.legend(loc=(0.85, 0.96-0.0273*legunit), prop={'size': 20})
ax.locator_params(nbins=6)

# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(20)

ax.title.set_fontsize(35)
ax.tick_params(direction='out', pad=3)
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f um'.format(1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f um'.format(1)))
plt.gca().set_xlim([-2, 2])
plt.gca().set_ylim([-2, 2])

# Save your figure
plt.savefig('{}.png'.format('Prelim_MOSAICTraj'), bbox_inches='tight')

plt.show()

In [42]:
# Now I need to do a similar process for Ben's data.  It shouldn't be that different that the above code.
# In fact, all I modified was the actual Excel spreadsheet.  I came up with a simple if statement to fill
# in the blank cells in his dataset, the changed the particle numbers to be sequential.  That's that!

trajectory1 = np.genfromtxt('../sample_data/Prelim_Ben_2DData.csv',
            delimiter =",")

In [43]:
trajectory1[1300, :]

array([  10.        ,  315.        ,    4.69439127,   12.97648608,    0.        ])

In [44]:
(final1, tots1) = prettify(trajectory1, 60, 60)
tots1

parts = tots1 - 1
ammon = final1[1]
counter = 1

while counter < parts:
    counter = counter + 1
    ammon = np.append(ammon, final1[counter], axis=0)

In [45]:
# Now for a plotting function, as modified from Testing_Tracking_Software

# Creates an array 'particles' that contains the particle number at each frame.
particles = ammon[:, 0]
position = ammon[:, 5:5+4]
total = int(max(particles))
total1 = total + 1
path = dict()

# Creates an array for each trajectory containing all xyz data
for num in range(1, total1):

    hold = np.where(particles == num)
    itindex = hold[0]
    min1 = min(itindex)
    max1 = max(itindex)
    path[num] = (position[min1:max1, :])

# Creates figure
fig = plt.figure(figsize=(24, 18), dpi=80)
ax = fig.add_subplot(111)
# ax.set_title('Particle Trajectories', x=0.5, y=1.15)

# Plots individual trajectories
for num in range(1, total1):

    ax.plot(path[num][:, 0], path[num][:, 1], label='Particle {}'.format(num))

axbox = ax.get_position()

legunit = max(ammon[:,0])
ax.legend(loc=(0.85, 0.96-0.0273*legunit), prop={'size': 20})
ax.locator_params(nbins=6)

# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(20)

ax.title.set_fontsize(35)
ax.tick_params(direction='out', pad=3)
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f um'.format(1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f um'.format(1)))
plt.gca().set_xlim([-30, 30])
plt.gca().set_ylim([-30, 30])

# Save your figure
plt.savefig('{}.png'.format('Prelim_BenTraj'), bbox_inches='tight')

plt.show()

In [ ]:
# And a final function to comapare their MSDs

# Creates an array 'particles' that contains the particle number at each frame.
particles = lehi[:, 0]
total = int(max(particles))
total1 = total + 1
rawtime = lehi[:, 15]
rawMSD = lehi[:, 17] #17 or 9
MSD = dict()
time = dict()

particles1 = ammon[:, 0]
totala = int(max(particles1))
total1a = totala + 1
rawtime1 = ammon[:, 15]
rawMSD1 = ammon[:, 17] #17 or 9
MSD1 = dict()
time1 = dict()

tr = 60

# Creates an array for each trajectory containing all xyz data
for num in range(1, total1):

    hold = np.where(particles == num)
    itindex = hold[0]
    min1 = min(itindex)
    max1 = max(itindex)
    MSD[num] = (rawMSD[min1+1:min1+tr])
    time[num] = (rawtime[min1+1:min1+tr])

MMSD = MSD[1]
for num in range(2, total1):
    MMSD = MMSD + MSD[num]

MMSD = MMSD/total1

for num in range(1, total1a):

    hold1 = np.where(particles1 == num)
    itindex1 = hold1[0]
    min1a = min(itindex1)
    max1a = max(itindex1)
    MSD1[num] = (rawMSD1[min1a+1:min1a+tr])
    time1[num] = (rawtime1[min1a+1:min1a+tr])

MMSD1 = MSD1[1]
for num in range(2, total1a):
    MMSD1 = MMSD1 + MSD1[num]

MMSD1 = MMSD1/total1a

# Creates figure
fig = plt.figure(figsize=(24, 18), dpi=80)
ax = fig.add_subplot(111)
# ax.set_title('Particle Trajectories', x=0.5, y=1.15)

ax.plot(time[1][:], MMSD[:], label='MOSAIC')
ax.plot(time1[1][:], MMSD1[:], label='Ben')

ax.legend(loc=(0.85, 0.85), prop={'size': 20})

# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
              ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(30)

ax.title.set_fontsize(55)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Deff')
ax.tick_params(direction='out', pad=5)
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(1)))

# Save your figure
plt.savefig('{}.png'.format('Prelim_CompareDeffBenMOSAIC'), bbox_inches='tight')

plt.show()

In [ ]:
# A couple issues that arose.  First, why do Ben's dataset and the MOSIAC dataset not end at the same point, when I
# defined them in my code to be the same? Frustrating.  Secondly, I will need to include some functionality to adjust
# units.  Right now the units for MSD are in pixels rather than microns.

# The plotting to different endpoints was a simple fix.  I define the range inside this code.  This would preferable be
# an input to a function.  I will make a variable for now.

In [ ]:
ammon[0:61, 15]

In [ ]:
lehi[0:61, 15]